In [1]:
import warnings
from dotenv import load_dotenv

load_dotenv()
warnings.filterwarnings('ignore')

### **Model 컴포넌트 실습**

In [2]:
# 랭체인 없이 모델을 활용할 때 작성하는 코드

from openai import OpenAI

client = OpenAI()
client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user", 
            "content": "2002년 월드컵 4강 국가 알려줘"
        }
    ]
)

ChatCompletion(id='chatcmpl-AMxWT55KG4cHr7v1qTt0mII9WHOdx', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='2002년 월드컵 4강에 진출한 국가는 브라질, 독일, 터키, 대한민국입니다. 브라질은 우승을 차지했고, 독일은 준우승, 터키는 3위, 대한민국은 4위를 차지했습니다.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1730035385, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=107, prompt_tokens=31, total_tokens=138, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))

In [3]:
# 랭체인을 활용한 모델 코드

from langchain_openai import ChatOpenAI
chat = ChatOpenAI(
    model_name = 'gpt-4o-mini'
)
chat.invoke("안녕~ 너를 소개해줄래?") # invoke함수를 통해 메세지를 보낼 수 있음

AIMessage(content='안녕하세요! 저는 AI 언어 모델인 ChatGPT입니다. 다양한 주제에 대해 대화하고, 질문에 답하며, 정보를 제공하는 역할을 하고 있습니다. 필요한 것이 있거나 궁금한 점이 있으면 언제든지 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 17, 'total_tokens': 70, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f59a81427f', 'finish_reason': 'stop', 'logprobs': None}, id='run-1b394501-5f31-4e3a-91e8-ac3c54bcdbff-0', usage_metadata={'input_tokens': 17, 'output_tokens': 53, 'total_tokens': 70})

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: Bad Request for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Empty request"}')
Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: Bad Request for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Empty request"}')


### **PromptTemplate 실습**

In [4]:
from langchain.prompts import PromptTemplate

# 템플릿으로 만들 문장을 만듬
# 중간에 괄호로 돼있는 {개수}와 {재료}가 사용자의 입력을 받아 동적으로 처리하는 곳
prompt= (
    PromptTemplate.from_template(
        """
        너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 {개수}개 추천하고, 
        그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.
        <재료>
        {재료}
        """
        )
    )

# prompt만 출력하면 '개수', '재료'는 입력을 받을꺼고, 나머지 부분은 템플릿 이라는 것을 볼 수 있다. (객체로 나오는걸 볼 수 있다.)
prompt 

PromptTemplate(input_variables=['개수', '재료'], template='\n        너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 {개수}추천하고, \n        그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.\n        <재료>\n        {재료}\n        ')

In [6]:
# 그래서 invoke에 위 매개변수를 적어서 처리하면 text로 string 결과값이 나온다.
prompt.invoke({"개수":4, "재료":"사과, 잼"})

StringPromptValue(text='\n        너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 4추천하고, \n        그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.\n        <재료>\n        사과, 잼\n        ')

### ChatPromptTemplate 실습

In [7]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
	#SystemMessage: 유용한 챗봇이라는 역할과 이름을 부여
        ("system", "You are a helpful AI bot. Your name is {name}."), 
    #HumanMessage와 AIMessage: 서로 안부를 묻고 답하는 대화 히스토리 주입
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
    #HumanMessage로 사용자가 입력한 프롬프트를 전달
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
print(messages) # 리스트 형태로 담기는 걸 볼 수 있다.

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'), HumanMessage(content='Hello, how are you doing?'), AIMessage(content="I'm doing well, thanks!"), HumanMessage(content='What is your name?')]


### **LCEL로 Chain 구축하기**

In [8]:
from langchain_core.output_parsers import StrOutputParser # StrOutputParser를 쓰게되면 ai의 content만 뽑아서 쓸 수 있다.
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

#프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}") # {topic}값을 주면 나한테 짧은 농담을 해주는 llm

#LLM 호출
model = ChatOpenAI(model="gpt-4o-mini")

#LCEL로 프롬프트템플릿-LLM-출력 파서 연결하기
chain = prompt | model | StrOutputParser()

#invoke함수로 chain 실행하기
chain.invoke({"topic": "ice cream"}) # 딱 응답 결과만 뽑아준것을 볼 수 있다.

'Why did the ice cream cone become a teacher? \n\nBecause it wanted to help kids get a little "scoop" of knowledge!'

In [11]:
#Chain 선언
model = ChatOpenAI(model="gpt-4o")
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

#Chain의 stream()함수를 통해 스트리밍 기능 추가
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True) # flush=True를 해줘야 실제로 스트림이 됨

Why don't bears wear shoes?

Because they prefer bear feet!

### **OutputParser 실습**
- OutputParser는 LLM으로부터 우리가 받은 답변을 원하는 형식으로 파싱하는 파서이다.
- 내가 원하는 형태대로 모델의 응답을 받아볼 수 있어서 이 후에 어떤 앱과 연결한다거나 서비스에 결합하기 굉장히 용이하다.
- 이번에 CSV OutputParser를 사용해보자


In [122]:
# CSV OutputParser를 사용하기 이전

from langchain_openai import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model_name="gpt-4o-mini",
                 temperature = 0)

#ChatPromptTemplate에 SystemMessage로 LLM의 역할과 출력 형식 지정
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "너는 영화 전문가 AI야. 사용자가 원하는 장르의 영화를 리스트 형태로 추천해줘."
                'ex) Query: SF영화 3개 추천해줘 / 답변: ["인터스텔라", "스페이스오디세이", "혹성탈출"]'
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
model = ChatOpenAI(model="gpt-4o-mini")
chain = chat_template | model
chain.invoke("액션")

AIMessage(content='["다크 나이트", "존 윅", "매드 맥스: 분노의 도로"]', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 72, 'total_tokens': 96}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-a0f3e3bf-1b95-46dc-83c9-607883166864-0', usage_metadata={'input_tokens': 72, 'output_tokens': 24, 'total_tokens': 96})

In [124]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

#CSV 파서 선언
output_parser = CommaSeparatedListOutputParser()

#CSV 파서 작동을 위한 형식 지정 프롬프트 로드
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [12]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

#CSV 파서 선언
output_parser = CommaSeparatedListOutputParser()

#CSV 파서 작동을 위한 형식 지정 프롬프트 로드
format_instructions = output_parser.get_format_instructions()

#프롬프트 템플릿의 partial_variables에 CSV 형식 지정 프롬프트 주입
prompt = PromptTemplate(
    template="List {subject}. answer in Korean \n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(model="gpt-4o-mini")

#프롬프트템플릿-모델-Output Parser를 체인으로 연결
chain = prompt | model | output_parser
chain.invoke({"subject": "공포 영화"})

['퀸', '곡성', '악마를 보았다', '써니', '새벽의 저주', '더 침묵', '인시디어스', '샤이닝', '헬레이저', '유전']

### JsonOutputParser
- Json형태에서는 데이터 구조를 먼저 정의를 해줘야 한다.
- 데이터 구조를 정의할 때는 pydantic 모델을 선호한다. 
- pydantic모델은 Json에서 해당 키 값이 어떤 형태의 타입을 가져야 하는지 명시를 해주고, 이 필드에 들어가야되는 값이 어떤 값인지 설명을 넣어준다.


In [128]:
from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# 데이터 구조를 정의합니다. 
class Country(BaseModel): # Country라는 클래스는 pydantic의 BaseModel을 가져왔기 때문에 
    continent: str = Field(description="사용자가 물어본 나라가 속한 대륙") # pydantic의 데이터 구조를 만들어 줄 수 있는 형태가 된다.
    population: str = Field(description="사용자가 물어본 나라의 인구(int 형식)") # 예) continent는 str타입을 가지고, "사용자가 물어본 나라가 속한 대륙"이라는 특성을 가지 키 값이 된다.
    

# JsonOutputParser를 설정하고 프롬프트 템플릿에 format_instructions를 삽입합니다.
parser = JsonOutputParser(pydantic_object=Country)

# PromptTemplate 정의
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 체인 연결
chain = prompt | model | parser

country_query = "아르헨티나는 어떤 나라야?"
chain.invoke({"query": country_query}) # 클래스로 정의한 Country 구조로 잘 출력되는걸 볼 수 있다.

{'continent': '남아메리카', 'population': '약 45,376,763'}

### **LCEL의 Runnable 객체에 대해 알아보기**

#### **RunnablePassthrough 알아보기**

In [130]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke("안녕하세요")

'안녕하세요'

In [132]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("""다음 한글 문장을 프랑스어로 번역해줘 {sentence}
French sentence: (print from here)""")

model = ChatOpenAI(model="gpt-4o-mini")

output_parser = StrOutputParser()

runnable_chain = {"sentence": RunnablePassthrough()} | prompt | model | output_parser
runnable_chain.invoke({"sentence": "그녀는 매일 아침 책을 읽습니다."})

'Elle lit un livre chaque matin.'

In [134]:
(RunnablePassthrough.assign(mult=lambda x: x["num"]*3)).invoke({"num":3})

{'num': 3, 'mult': 9}

In [136]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'extra': {'num': 1, 'mult': 3}, 'modified': 2}

#### **RunnableLambda 알아보기**
- 사용자가 정의한 함수에 Runnable객체의 기능을 넣어주는 것

In [138]:
# 사용방법 예시) 사용자가 입력한 값에다 :) 웃음표시를 string형태로 더해주는 함수를 만듬
def add_smile(x):
    return x + ":)"

In [140]:
from langchain_core.runnables import RunnableLambda

# 사용방법 예시) 만든 함수를 Runnable객체에 넣어주면 Runnable객체의 형태로 이 함수를 활용할 수 있음
add_smile = RunnableLambda(add_smile)

In [14]:
# 실제로 사용해보기
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

# 사용자에게 토픽에대한 입력을 받으면 그에대한 역사를 세문장으로 설명하는 프롬프트
prompt_str = "{topic}의 역사에 대해 세문장으로 설명해주세요."
prompt = ChatPromptTemplate.from_template(prompt_str) # ChatPromptTemplate 설정

model = ChatOpenAI(model_name = 'gpt-4o-mini') # 모델 설정

output_parser = StrOutputParser() # 아웃파서 설정

chain = prompt | model | output_parser # 체인 연결

In [19]:
from langchain_core.runnables import RunnableLambda

#  사용자가 입력한 값 + "들어주셔서 감사합니다 :)" 를 더해주는 함수를 만듬
def add_thank(x):
    return x + " 들어주셔서 감사합니다 :)"

add_thank = RunnableLambda(add_thank)

In [20]:
# 프롬프트 작성하여 모델에 전달 -> 받은 프롬프트 모델이 답변 -> 답변을 str형태로 값 추출 -> 그 값을 add_thank의 매개변수로 넣어 실행
chain = prompt | model | output_parser | add_thank
chain.invoke("반도체")

'반도체의 역사는 20세기 초반에 시작되어, 1947년 벨 연구소의 과학자들이 트랜지스터를 발명함으로써 본격적으로 발전하게 되었습니다. 이후 1960년대에는 집적회로(IC)의 개발로 반도체 기술이 급속도로 발전하였고, 이는 컴퓨터와 전자기기의 혁신을 이끌었습니다. 21세기에는 모바일 기기와 인공지능, 자율주행차 등 다양한 분야에서 반도체의 중요성이 더욱 커지며, 글로벌 경제의 핵심 산업으로 자리 잡고 있습니다. 들어주셔서 감사합니다 :)'

#### **RunnableParallel 알아보기**
- 여러가지의 계산로직을 함께 담으면 병렬로 실행해준다.

In [148]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(), # RunnableParallel의 들어온 값을 그대로 내보냄
    modified=lambda x: x["num"] + 1, # 그 들어온 값에 num에 해당 하는 것은 1을 더한걸 modified에 할당
)

# 실행해보면 passed로 설정한 값은 num은 1이다 라는 딕셔너리(객체)가 그대로 출력돼 있다. 'passed': {'num': 1}
# modified는 병렬로 별도의 로직이 계산이 되었기 때문에 passed에 값으로 들어간게 아닌 modified의 값에 2가 들어간걸 볼 수 있다. (num + 1) = 2
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'modified': 2}

In [150]:
# modified에 아까 위에 선언한 add_thank함수를 정의 (들어주셔서 감사합니다 :))
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=add_thank,
)

# passed는 들어온값 그대로 출력 modified는 들어온값("안녕하세요") + add_thank("들어주셔서 감사합니다 :)")
runnable.invoke("안녕하세요")

{'passed': '안녕하세요', 'modified': '안녕하세요 들어주셔서 감사합니다 :)'}

### RunnableParallel을 통해서 여러가지의 결과물을 한꺼번에 받아오기

In [152]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

model = ChatOpenAI(model = 'gpt-4o-mini', max_tokens = 128, temperature = 0) # 답변이 너무 길어지지 않게 토큰 정의

# ChatPromptTemplate을 각각 정의
history_prompt = ChatPromptTemplate.from_template("{topic}가 무엇의 약자인지 알려주세요.")
celeb_prompt = ChatPromptTemplate.from_template("{topic} 분야의 유명인사 3명의 이름만 알려주세요.")

output_parser = StrOutputParser()

# 각각 체인을 정의 (사용자의 입력이 들어오면 각각 다른 결과물이 나올것임)
history_chain = history_prompt | model | output_parser
celeb_chain = celeb_prompt | model | output_parser

# RunnableParallel을 선언 후 안에 위에 정의한 체인을 각각 넣어줌
map_chain = RunnableParallel(history=history_chain, celeb=celeb_chain)

# 사용자 입력 값 "AI" 토픽을 전해서 각 체인의 값이 출력됨
result = map_chain.invoke({"topic": "AI"})

result
# 결과물이 딕셔너리에 담겨 출력이 되는걸 불 수 있음.
# {
#  'history': '히스토리 체인 결과물 출력',
#  'celeb': '셀럽 체인 결과물 출력'
# }


{'history': 'AI는 "Artificial Intelligence"의 약자로, 한국어로는 "인공지능"이라고 합니다. 인공지능은 컴퓨터 시스템이 인간의 지능을 모방하여 학습, 추론, 문제 해결, 이해 등의 작업을 수행할 수 있도록 하는 기술을 의미합니다.',
 'celeb': '1. 앨런 튜링 (Alan Turing)\n2. 제프리 힌튼 (Geoffrey Hinton)\n3. 얀 르쿤 (Yann LeCun)'}